Kaggle daki Chest X-Rays dataset ile Pneumonia hastası olup olmadığını belirlemeye en uygun modeli belirleyeceğiz. 

In [1]:
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


Aşağıda da gördüğümüz gibi iki ana dizin test train i kullanacağız. 
Her ikisinin altında da iki tane NORMAL ve PNEUMONIA olarak dosya yer almaktadır. 
Oncelikle Train ve Test için ayrı ayrı NORMAL VE PNEUMONIA dosyalarını rastsal birleştireceğiz.
y yi de hasta olanlar 1 ve saglıklılar 0 olarak atayacağız.

In [2]:

path = "/Users/Mehtap_Labtop/Downloads/chest-xray-pneumonia/chest_xray/"
dir_list = os.listdir(path) 
print(dir_list) 

['chest_xray', 'test', 'train', 'val', '__MACOSX']


In [9]:

dir_list = os.listdir("/Users/Mehtap_Labtop/Downloads/chest-xray-pneumonia/chest_xray/train/NORMAL/") 

Xtrain ve Ytrain olustururken her bir goruntu arrayın boyutları yuksek olduğundan bu değeri (400,300) e sabitledim.

In [5]:
X_egitim_resize1=list()
y_egitim1=list()
for i in range(len(dir_list)):
    img_normal = load_img('/Users/Mehtap_Labtop/Downloads/chest-xray-pneumonia/chest_xray/train/NORMAL/' + dir_list[i])
    np_im_n = np.array(img_normal)
    newsize = (400, 300) 
    im1 = img_normal.resize(newsize) 
    np_im1_n = np.array(im1)
    X_egitim_resize1.append(np_im1_n)
    y_egitim1.append(0)
len(y_egitim1)

1341

In [6]:
dir_list_p = os.listdir("/Users/Mehtap_Labtop/Downloads/chest-xray-pneumonia/chest_xray/train/PNEUMONIA/") 

X_egitim_resize2=list()
y_egitim2=list()
for i in range(len(dir_list_p)):
    img_PNEUMONIA = load_img('/Users/Mehtap_Labtop/Downloads/chest-xray-pneumonia/chest_xray/train/PNEUMONIA/' + dir_list_p[i])
    newsize = (400, 300) 
    imp1 = img_PNEUMONIA.resize(newsize) 
    np_im1_p = np.array(imp1)
    X_egitim_resize2.append(np_im1_p)
    y_egitim2.append(1)
len(y_egitim2)

3875

Train degerlerini toplarken once saglıklı ve hastalıklı verileri arka arkaya birleştiriyoruz.
Sonrasında rastsal dagıtarak yenıden veriseti olusturacağız.

In [11]:
y_egıtım=np.concatenate((y_egitim1, y_egitim2), axis=0)

In [12]:
from keras.utils import to_categorical
y_egitim_kategorik = to_categorical(y_egıtım, 2)
y_egitim_kategorik.shape

(5216, 2)

In [13]:
X_egitim_resize=np.concatenate((X_egitim_resize1, X_egitim_resize2), axis=0)

In [15]:
X_egitim_resize.shape

(5216, 300, 400, 3)

Şimdi verisetini, sağlıklı sonuç alabilmek için, rastsal olarak kendi içinde karıstırıyoruz.

In [16]:
import random
X_egitim_random=list()
y_egitim_random=list()
liste = list(range(0,5216))
while len(liste)>0:
    a=random.choice(liste)
    X_egitim_random.append(X_egitim_resize[a])
    y_egitim_random.append(y_egitim_kategorik[a])
    liste.remove(a)

In [17]:
X_egitim_random = np.array(X_egitim_random)
y_egitim_random = np.array(y_egitim_random)

Yukarıdaki normal ve hastalıklı veriyi birleştirip, rastsal duzenleme işlemini aşağıda test verileri için yaptım

In [18]:
dir_list_n = os.listdir("/Users/Mehtap_Labtop/Downloads/chest-xray-pneumonia/chest_xray/test/NORMAL/") 

X_test_resize1=list()
y_test1=list()
for i in range(len(dir_list_n)):
    img_normal = load_img('/Users/Mehtap_Labtop/Downloads/chest-xray-pneumonia/chest_xray/test/NORMAL/' + dir_list_n[i])
    np_im_n = np.array(img_normal)
    newsize = (400, 300) 
    im1 = img_normal.resize(newsize) 
    np_im1_n = np.array(im1)
    X_test_resize1.append(np_im1_n)
    y_test1.append(0)

In [19]:
dir_list_p = os.listdir("/Users/Mehtap_Labtop/Downloads/chest-xray-pneumonia/chest_xray/test/PNEUMONIA/") 

X_test_resize2=list()
y_test2=list()
for i in range(len(dir_list_p)):
    img_normal = load_img('/Users/Mehtap_Labtop/Downloads/chest-xray-pneumonia/chest_xray/test/PNEUMONIA/' + dir_list_p[i])
    np_im_n = np.array(img_normal)
    newsize = (400, 300) 
    im1 = img_normal.resize(newsize) 
    np_im1_n = np.array(im1)
    X_test_resize2.append(np_im1_n)
    y_test2.append(1)


In [20]:
y_test=np.concatenate((y_test1, y_test2), axis=0)
y_test_kategorik = to_categorical(y_test, 2)

X_test_resize=np.concatenate((X_test_resize1, X_test_resize2), axis=0)


In [21]:

X_test_random=list()
y_test_random=list()
liste = list(range(0,624))
while len(liste)>0:
    a=random.choice(liste)
    X_test_random.append(X_test_resize[a])
    y_test_random.append(y_test_kategorik[a])
    liste.remove(a)
X_test_random = np.array(X_test_random)
y_test_random = np.array(y_test_random)

In [22]:
img_satır = X_egitim_random.shape[1]
img_sutun = X_egitim_random.shape[2]
kanal_sayisi = X_egitim_random.shape[3]
print(img_satır, img_sutun, kanal_sayisi)

300 400 3


### Aşağıda çeşitli modeller tasarlayarak değerlendirme sonuçları çıkartıldı

In [24]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

model = Sequential()
model.add(Conv2D(filters = 8, kernel_size=5, activation='relu', strides = (2,2), 
               input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()

model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_random, y_egitim_random, 
          validation_split=0.2, 
          epochs=3, batch_size=32)

dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 148, 198, 8)       608       
_________________________________________________________________
flatten_2 (Flatten)          (None, 234432)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 468866    
Total params: 469,474
Trainable params: 469,474
Non-trainable params: 0
_________________________________________________________________
Train on 4172 samples, validate on 1044 samples
Epoch 1/3
4172/4172 [==============================] - 35s 8ms/step - loss: 369.1654 - accuracy: 0.8138 - val_loss: 0.9902 - val_accuracy: 0.8678
Epoch 2/3
4172/4172 [==============================] - 33s 8ms/step - loss: 0.2836 - accuracy: 0.9506 - val_loss: 0.9520 - val_accuracy: 0.8726
Epoch 3/3
624/624 [=====================

In [22]:
model = Sequential()
model.add(Conv2D(filters = 8, kernel_size=5, activation='relu', strides = (2,2), 
               input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()

model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_random, y_egitim_random, 
          validation_split=0.2, 
          epochs=6, batch_size=32)

dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 148, 198, 8)       608       
_________________________________________________________________
flatten_2 (Flatten)          (None, 234432)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 468866    
Total params: 469,474
Trainable params: 469,474
Non-trainable params: 0
_________________________________________________________________
Train on 4172 samples, validate on 1044 samples
Epoch 1/6
4172/4172 [==============================] - 29s 7ms/step - loss: 498.9709 - accuracy: 0.7677 - val_loss: 1.7185 - val_accuracy: 0.8352
Epoch 2/6
4172/4172 [==============================] - 30s 7ms/step - loss: 0.4896 - accuracy: 0.9221 - val_loss: 1.4670 - val_accuracy: 0.8659
Epoch 3/6
4172/4172 [===================

In [23]:
import matplotlib.pyplot as plt
from keras.layers import MaxPool2D, AvgPool2D, Dropout
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(2))
    
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_resize, y_egitim_kategorik, 
          validation_split=0.2, 
          epochs=6, batch_size=32)


dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 298, 398, 16)      448       
_________________________________________________________________
batch_normalization_1 (Batch (None, 298, 398, 16)      64        
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 296, 396, 16)      2320      
_________________________________________________________________
batch_normalization_2 (Batch (None, 296, 396, 16)      64        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 148, 198, 16)      0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 468864)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                

In [24]:
import matplotlib.pyplot as plt
from keras.layers import MaxPool2D, AvgPool2D, Dropout
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(MaxPool2D(2))
    
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_resize, y_egitim_kategorik, 
          validation_split=0.2, 
          epochs=10, batch_size=32)


dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 298, 398, 16)      448       
_________________________________________________________________
batch_normalization_3 (Batch (None, 298, 398, 16)      64        
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 296, 396, 16)      2320      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 148, 198, 16)      0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 468864)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 937730    
Total params: 940,562
Trainable params: 940,530
Non-trainable params: 32
_______________________________________________

In [25]:
import matplotlib.pyplot as plt
from keras.layers import MaxPool2D, AvgPool2D, Dropout
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(Dropout(0.50))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(MaxPool2D(2))
model.add(Dropout(0.50))
    
model.add(Conv2D(filters = 8, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_resize, y_egitim_kategorik, 
          validation_split=0.2, 
          epochs=10, batch_size=32)


dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 298, 398, 16)      448       
_________________________________________________________________
dropout_1 (Dropout)          (None, 298, 398, 16)      0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 298, 398, 16)      64        
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 296, 396, 16)      2320      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 148, 198, 16)      0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 148, 198, 16)      0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 146, 196, 8)      

In [26]:
import matplotlib.pyplot as plt
from keras.layers import MaxPool2D, AvgPool2D, Dropout
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(MaxPool2D(2))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(2))
    
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_resize, y_egitim_kategorik, 
          validation_split=0.2, 
          epochs=6, batch_size=32)


dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 298, 398, 16)      448       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 149, 199, 16)      0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 149, 199, 16)      64        
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 147, 197, 16)      2320      
_________________________________________________________________
batch_normalization_6 (Batch (None, 147, 197, 16)      64        
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 73, 98, 16)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 114464)           

In [27]:
import matplotlib.pyplot as plt
from keras.layers import MaxPool2D, AvgPool2D, Dropout
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(Dropout(0.50))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(2))
    
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_resize, y_egitim_kategorik, 
          validation_split=0.2, 
          epochs=6, batch_size=32)


dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 298, 398, 16)      448       
_________________________________________________________________
dropout_3 (Dropout)          (None, 298, 398, 16)      0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 298, 398, 16)      64        
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 296, 396, 16)      2320      
_________________________________________________________________
batch_normalization_8 (Batch (None, 296, 396, 16)      64        
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 148, 198, 16)      0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 468864)           

In [28]:
import matplotlib.pyplot as plt
from keras.layers import MaxPool2D, AvgPool2D, Dropout
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(2))
    
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_resize, y_egitim_kategorik, 
          validation_split=0.2, 
          epochs=6, batch_size=32)


dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 298, 398, 16)      448       
_________________________________________________________________
batch_normalization_9 (Batch (None, 298, 398, 16)      64        
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 296, 396, 16)      2320      
_________________________________________________________________
batch_normalization_10 (Batc (None, 296, 396, 16)      64        
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 294, 394, 16)      2320      
_________________________________________________________________
batch_normalization_11 (Batc (None, 294, 394, 16)      64        
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 147, 197, 16)     

In [29]:
import matplotlib.pyplot as plt
from keras.layers import MaxPool2D, AvgPool2D, Dropout
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(Dropout(0.50))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(2))
    
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_resize, y_egitim_kategorik, 
          validation_split=0.2, 
          epochs=6, batch_size=32)


dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 298, 398, 16)      448       
_________________________________________________________________
dropout_4 (Dropout)          (None, 298, 398, 16)      0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 298, 398, 16)      64        
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 296, 396, 16)      2320      
_________________________________________________________________
batch_normalization_13 (Batc (None, 296, 396, 16)      64        
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 148, 198, 16)      0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 468864)           

In [30]:
import matplotlib.pyplot as plt
from keras.layers import MaxPool2D, AvgPool2D, Dropout
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(Dropout(0.50))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(2))
    
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_resize, y_egitim_kategorik, 
          validation_split=0.2, 
          epochs=10, batch_size=32)


dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 298, 398, 16)      448       
_________________________________________________________________
dropout_5 (Dropout)          (None, 298, 398, 16)      0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 298, 398, 16)      64        
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 296, 396, 16)      2320      
_________________________________________________________________
batch_normalization_15 (Batc (None, 296, 396, 16)      64        
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 148, 198, 16)      0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 468864)          

In [31]:
#5
import matplotlib.pyplot as plt
from keras.layers import MaxPool2D, AvgPool2D, Dropout
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(Dropout(0.50))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(BatchNormalization())

model.add(Conv2D(filters = 16, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(2))
    
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_resize, y_egitim_kategorik, 
          validation_split=0.2, 
          epochs=6, batch_size=32)


dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 298, 398, 16)      448       
_________________________________________________________________
dropout_6 (Dropout)          (None, 298, 398, 16)      0         
_________________________________________________________________
batch_normalization_16 (Batc (None, 298, 398, 16)      64        
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 296, 396, 16)      2320      
_________________________________________________________________
batch_normalization_17 (Batc (None, 296, 396, 16)      64        
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 294, 394, 16)      2320      
_________________________________________________________________
batch_normalization_18 (Batc (None, 294, 394, 16)    

In [32]:
#6
import matplotlib.pyplot as plt
from keras.layers import MaxPool2D, AvgPool2D, Dropout
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(Dropout(0.50))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(2))
    
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_resize, y_egitim_kategorik, 
          validation_split=0.2, 
          epochs=6, batch_size=32)


dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 298, 398, 16)      448       
_________________________________________________________________
dropout_7 (Dropout)          (None, 298, 398, 16)      0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 298, 398, 16)      64        
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 296, 396, 16)      2320      
_________________________________________________________________
batch_normalization_20 (Batc (None, 296, 396, 16)      64        
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 148, 198, 16)      0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 468864)          

In [33]:
#7
import matplotlib.pyplot as plt
from keras.layers import MaxPool2D, AvgPool2D, Dropout
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(MaxPool2D(2))
model.add(BatchNormalization())
model.add(Conv2D(filters = 8, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(2))
    
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_resize, y_egitim_kategorik, 
          validation_split=0.2, 
          epochs=6, batch_size=32)


dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 298, 398, 16)      448       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 149, 199, 16)      0         
_________________________________________________________________
batch_normalization_21 (Batc (None, 149, 199, 16)      64        
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 147, 197, 8)       1160      
_________________________________________________________________
batch_normalization_22 (Batc (None, 147, 197, 8)       32        
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 73, 98, 8)         0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 57232)           

In [34]:
#iyi gibi
import matplotlib.pyplot as plt
from keras.layers import MaxPool2D, AvgPool2D, Dropout
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(MaxPool2D(2))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(MaxPool2D(2))
    
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_resize, y_egitim_kategorik, 
          validation_split=0.2, 
          epochs=6, batch_size=32)


dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 298, 398, 16)      448       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 149, 199, 16)      0         
_________________________________________________________________
batch_normalization_23 (Batc (None, 149, 199, 16)      64        
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 147, 197, 16)      2320      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 73, 98, 16)        0         
_________________________________________________________________
flatten_14 (Flatten)         (None, 114464)            0         
_________________________________________________________________
dense_14 (Dense)             (None, 2)               

In [35]:
#9
import matplotlib.pyplot as plt
from keras.layers import MaxPool2D, AvgPool2D, Dropout
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(MaxPool2D(2))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(2))
    
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_resize, y_egitim_kategorik, 
          validation_split=0.2, 
          epochs=10, batch_size=32)


dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 298, 398, 16)      448       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 149, 199, 16)      0         
_________________________________________________________________
batch_normalization_24 (Batc (None, 149, 199, 16)      64        
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 147, 197, 16)      2320      
_________________________________________________________________
batch_normalization_25 (Batc (None, 147, 197, 16)      64        
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 73, 98, 16)        0         
_________________________________________________________________
flatten_15 (Flatten)         (None, 114464)          

## Güzel Sonuç

In [37]:
## Güzel Sonuç
import matplotlib.pyplot as plt
from keras.layers import MaxPool2D, AvgPool2D, Dropout
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(Dropout(0.20))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(MaxPool2D(2))
model.add(Dropout(0.20))
    
model.add(Conv2D(filters = 8, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_resize, y_egitim_kategorik, 
          validation_split=0.2, 
          epochs=10, batch_size=32)


dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_34 (Conv2D)           (None, 298, 398, 16)      448       
_________________________________________________________________
dropout_8 (Dropout)          (None, 298, 398, 16)      0         
_________________________________________________________________
batch_normalization_28 (Batc (None, 298, 398, 16)      64        
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 296, 396, 16)      2320      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 148, 198, 16)      0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 148, 198, 16)      0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 146, 196, 8)     

In [38]:
#12
import matplotlib.pyplot as plt
from keras.layers import MaxPool2D, AvgPool2D, Dropout
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(Dropout(0.20))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(2))
    
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_resize, y_egitim_kategorik, 
          validation_split=0.2, 
          epochs=6, batch_size=32)


dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 298, 398, 16)      448       
_________________________________________________________________
dropout_10 (Dropout)         (None, 298, 398, 16)      0         
_________________________________________________________________
batch_normalization_29 (Batc (None, 298, 398, 16)      64        
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 296, 396, 16)      2320      
_________________________________________________________________
batch_normalization_30 (Batc (None, 296, 396, 16)      64        
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 148, 198, 16)      0         
_________________________________________________________________
flatten_18 (Flatten)         (None, 468864)          

## Güzel Sonuç

In [39]:

import matplotlib.pyplot as plt
from keras.layers import MaxPool2D, AvgPool2D, Dropout
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(Conv2D(filters = 16, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(MaxPool2D(2))
    
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_resize, y_egitim_kategorik, 
          validation_split=0.2, 
          epochs=6, batch_size=32)


dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_39 (Conv2D)           (None, 298, 398, 16)      448       
_________________________________________________________________
batch_normalization_31 (Batc (None, 298, 398, 16)      64        
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 296, 396, 16)      2320      
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 294, 394, 16)      2320      
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 147, 197, 16)      0         
_________________________________________________________________
flatten_19 (Flatten)         (None, 463344)            0         
_________________________________________________________________
dense_19 (Dense)             (None, 2)               

In [40]:
#14
import matplotlib.pyplot as plt
from keras.layers import MaxPool2D, AvgPool2D, Dropout
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 8, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(Dropout(0.20))
model.add(BatchNormalization())
model.add(Conv2D(filters = 8, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(2))
    
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_resize, y_egitim_kategorik, 
          validation_split=0.2, 
          epochs=6, batch_size=32)


dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_42 (Conv2D)           (None, 298, 398, 8)       224       
_________________________________________________________________
dropout_11 (Dropout)         (None, 298, 398, 8)       0         
_________________________________________________________________
batch_normalization_32 (Batc (None, 298, 398, 8)       32        
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 296, 396, 8)       584       
_________________________________________________________________
batch_normalization_33 (Batc (None, 296, 396, 8)       32        
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 148, 198, 8)       0         
_________________________________________________________________
flatten_20 (Flatten)         (None, 234432)          

## Güzel Sonuç

In [42]:

import matplotlib.pyplot as plt
from keras.layers import MaxPool2D, AvgPool2D, Dropout
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(Dropout(0.20))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(2))
    
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_resize, y_egitim_kategorik, 
          validation_split=0.2, 
          epochs=10, batch_size=32)


dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_46 (Conv2D)           (None, 298, 398, 16)      448       
_________________________________________________________________
dropout_13 (Dropout)         (None, 298, 398, 16)      0         
_________________________________________________________________
batch_normalization_36 (Batc (None, 298, 398, 16)      64        
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 296, 396, 16)      2320      
_________________________________________________________________
batch_normalization_37 (Batc (None, 296, 396, 16)      64        
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 148, 198, 16)      0         
_________________________________________________________________
flatten_22 (Flatten)         (None, 468864)          

In [43]:
#16
import matplotlib.pyplot as plt
from keras.layers import MaxPool2D, AvgPool2D, Dropout
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(Dropout(0.20))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(Conv2D(filters = 16, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(2))
    
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_resize, y_egitim_kategorik, 
          validation_split=0.2, 
          epochs=10, batch_size=32)


dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_48 (Conv2D)           (None, 298, 398, 16)      448       
_________________________________________________________________
dropout_14 (Dropout)         (None, 298, 398, 16)      0         
_________________________________________________________________
batch_normalization_38 (Batc (None, 298, 398, 16)      64        
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 296, 396, 16)      2320      
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 294, 394, 16)      2320      
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 292, 392, 16)      2320      
_________________________________________________________________
batch_normalization_39 (Batc (None, 292, 392, 16)    

In [44]:
import matplotlib.pyplot as plt
from keras.layers import MaxPool2D, AvgPool2D, Dropout
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(Dropout(0.20))
model.add(BatchNormalization())
model.add(Conv2D(filters = 8, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(MaxPool2D(2))
model.add(Dropout(0.20))
    
model.add(Conv2D(filters = 8, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_resize, y_egitim_kategorik, 
          validation_split=0.2, 
          epochs=10, batch_size=32)


dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_52 (Conv2D)           (None, 298, 398, 16)      448       
_________________________________________________________________
dropout_15 (Dropout)         (None, 298, 398, 16)      0         
_________________________________________________________________
batch_normalization_40 (Batc (None, 298, 398, 16)      64        
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 296, 396, 8)       1160      
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 148, 198, 8)       0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 148, 198, 8)       0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 146, 196, 8)     

In [46]:
import matplotlib.pyplot as plt
from keras.layers import MaxPool2D, AvgPool2D, Dropout
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(Dropout(0.20))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(Conv2D(filters = 8, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(MaxPool2D(2))
model.add(Dropout(0.20))
    
model.add(Conv2D(filters = 8, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_resize, y_egitim_kategorik, 
          validation_split=0.2, 
          epochs=10, batch_size=32)


dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_59 (Conv2D)           (None, 298, 398, 16)      448       
_________________________________________________________________
dropout_19 (Dropout)         (None, 298, 398, 16)      0         
_________________________________________________________________
batch_normalization_42 (Batc (None, 298, 398, 16)      64        
_________________________________________________________________
conv2d_60 (Conv2D)           (None, 296, 396, 16)      2320      
_________________________________________________________________
conv2d_61 (Conv2D)           (None, 294, 394, 8)       1160      
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 147, 197, 8)       0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 147, 197, 8)     

## Güzel Sonuç

In [47]:

import matplotlib.pyplot as plt
from keras.layers import MaxPool2D, AvgPool2D, Dropout
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(Dropout(0.20))
model.add(BatchNormalization())
model.add(Conv2D(filters = 8, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(Conv2D(filters = 8, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(MaxPool2D(2))
model.add(Dropout(0.20))
    
model.add(Conv2D(filters = 8, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_resize, y_egitim_kategorik, 
          validation_split=0.2, 
          epochs=10, batch_size=32)


dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_63 (Conv2D)           (None, 298, 398, 16)      448       
_________________________________________________________________
dropout_21 (Dropout)         (None, 298, 398, 16)      0         
_________________________________________________________________
batch_normalization_43 (Batc (None, 298, 398, 16)      64        
_________________________________________________________________
conv2d_64 (Conv2D)           (None, 296, 396, 8)       1160      
_________________________________________________________________
conv2d_65 (Conv2D)           (None, 294, 394, 8)       584       
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 147, 197, 8)       0         
_________________________________________________________________
dropout_22 (Dropout)         (None, 147, 197, 8)     

## Güzel Sonuç

In [48]:

import matplotlib.pyplot as plt
from keras.layers import MaxPool2D, AvgPool2D, Dropout
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(Dropout(0.20))
model.add(BatchNormalization())
model.add(Conv2D(filters = 8, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(MaxPool2D(2))
model.add(Dropout(0.20))
model.add(Conv2D(filters = 8, kernel_size=3,strides=(1, 1), activation='relu'))   
model.add(Conv2D(filters = 8, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_resize, y_egitim_kategorik, 
          validation_split=0.2, 
          epochs=10, batch_size=32)


dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_67 (Conv2D)           (None, 298, 398, 16)      448       
_________________________________________________________________
dropout_23 (Dropout)         (None, 298, 398, 16)      0         
_________________________________________________________________
batch_normalization_44 (Batc (None, 298, 398, 16)      64        
_________________________________________________________________
conv2d_68 (Conv2D)           (None, 296, 396, 8)       1160      
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 148, 198, 8)       0         
_________________________________________________________________
dropout_24 (Dropout)         (None, 148, 198, 8)       0         
_________________________________________________________________
conv2d_69 (Conv2D)           (None, 146, 196, 8)     

In [49]:
import matplotlib.pyplot as plt
from keras.layers import MaxPool2D, AvgPool2D, Dropout
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(Dropout(0.10))
model.add(BatchNormalization())
model.add(Conv2D(filters = 8, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(MaxPool2D(2))
model.add(Dropout(0.10))
model.add(Conv2D(filters = 8, kernel_size=3,strides=(1, 1), activation='relu'))   
model.add(Dropout(0.10))
model.add(Conv2D(filters = 8, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_resize, y_egitim_kategorik, 
          validation_split=0.2, 
          epochs=10, batch_size=32)


dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_71 (Conv2D)           (None, 298, 398, 16)      448       
_________________________________________________________________
dropout_25 (Dropout)         (None, 298, 398, 16)      0         
_________________________________________________________________
batch_normalization_45 (Batc (None, 298, 398, 16)      64        
_________________________________________________________________
conv2d_72 (Conv2D)           (None, 296, 396, 8)       1160      
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 148, 198, 8)       0         
_________________________________________________________________
dropout_26 (Dropout)         (None, 148, 198, 8)       0         
_________________________________________________________________
conv2d_73 (Conv2D)           (None, 146, 196, 8)     

In [50]:
#iyi gibi
import matplotlib.pyplot as plt
from keras.layers import MaxPool2D, AvgPool2D, Dropout
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(Dropout(0.10))
model.add(BatchNormalization())
model.add(Conv2D(filters = 8, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(MaxPool2D(2))
model.add(Dropout(0.10))
model.add(Conv2D(filters = 8, kernel_size=3,strides=(1, 1), activation='relu'))   
model.add(MaxPool2D(2))
model.add(Dropout(0.10))
model.add(Conv2D(filters = 8, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_resize, y_egitim_kategorik, 
          validation_split=0.2, 
          epochs=10, batch_size=32)


dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_75 (Conv2D)           (None, 298, 398, 16)      448       
_________________________________________________________________
dropout_28 (Dropout)         (None, 298, 398, 16)      0         
_________________________________________________________________
batch_normalization_46 (Batc (None, 298, 398, 16)      64        
_________________________________________________________________
conv2d_76 (Conv2D)           (None, 296, 396, 8)       1160      
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 148, 198, 8)       0         
_________________________________________________________________
dropout_29 (Dropout)         (None, 148, 198, 8)       0         
_________________________________________________________________
conv2d_77 (Conv2D)           (None, 146, 196, 8)     

In [51]:
import matplotlib.pyplot as plt
from keras.layers import MaxPool2D, AvgPool2D, Dropout
from keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size=3, activation='relu', 
                 input_shape=(img_satır, img_sutun,kanal_sayisi)))
model.add(Dropout(0.10))
model.add(BatchNormalization())
model.add(Conv2D(filters = 8, kernel_size=3,strides=(1, 1), activation='relu'))
model.add(MaxPool2D(2))
model.add(Dropout(0.10))
model.add(Conv2D(filters = 8, kernel_size=3,strides=(1, 1), activation='relu'))   
model.add(MaxPool2D(2))
model.add(Dropout(0.10))
model.add(Conv2D(filters = 8, kernel_size=3,strides=(1, 1), activation='relu'))  
model.add(Conv2D(filters = 8, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
                 
model.summary()
    
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_egitim_resize, y_egitim_kategorik, 
          validation_split=0.2, 
          epochs=10, batch_size=32)


dogruluk_egitim = model.evaluate(X_egitim_random, y_egitim_random)[1]
dogruluk_test = model.evaluate(X_test_random, y_test_random)[1]

print("\nEğitim kümesi doğruluk değeri : {:.3f}".format(dogruluk_egitim))
print("Test kümesi doğruluk değeri   : {:.3f}".format(dogruluk_test))

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_79 (Conv2D)           (None, 298, 398, 16)      448       
_________________________________________________________________
dropout_31 (Dropout)         (None, 298, 398, 16)      0         
_________________________________________________________________
batch_normalization_47 (Batc (None, 298, 398, 16)      64        
_________________________________________________________________
conv2d_80 (Conv2D)           (None, 296, 396, 8)       1160      
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 148, 198, 8)       0         
_________________________________________________________________
dropout_32 (Dropout)         (None, 148, 198, 8)       0         
_________________________________________________________________
conv2d_81 (Conv2D)           (None, 146, 196, 8)     

Modeller arasında "Güzel Sonuç başlığı" olarak eklediğim model en uygun model gözükmektedir.